In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
import torchsummary

import pandas as pd
import numpy as np
import cv2
import json
import os
import shutil
import glob
import time
import copy
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# !git clone https://github.com/prajnasb/observations

In [4]:
# !ls observations/experiements/dest_folder/

In [5]:
# !cp -r /content/observations/experiements/dest_folder/test /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset
# !cp -r /content/observations/experiements/dest_folder/train /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset
# !cp -r /content/observations/experiements/dest_folder/val /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset
# !cp -r /content/observations/experiements/dest_folder/test.csv /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset
# !cp -r /content/observations/experiements/dest_folder/train.csv /content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset

In [22]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mjbyeon","key":"2753818815500b5d2469eb0f6a12bf5c"}'}

In [23]:
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [25]:
!kaggle datasets download -d omkargurav/face-mask-dataset

 99% 161M/163M [00:08<00:00, 24.6MB/s]
100% 163M/163M [00:08<00:00, 20.9MB/s]


In [ ]:
!unzip /content/face-mask-dataset.zip

In [35]:
def split(img_list, test_count, train_path, test_path):
    test_files = []
    for i in random.sample(img_list, test_count):
        test_files.append(i)
    
    train_files = [x for x in img_list if x not in test_files]

    for k in train_files:
        shutil.copy(k, train_path)
    for c in test_files:
        shutil.copy(c, test_path)

In [36]:
with_mask = glob.glob('/content/data/with_mask/*')
without_mask = glob.glob('/content/data/without_mask/*')

with_mask_train_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/with_mask'
with_mask_test_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/with_mask'

without_mask_train_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/without_mask'
without_mask_test_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/without_mask'

with_mask_test_count = round(len(with_mask)*0.2)
without_mask_test_count = round(len(without_mask)*0.2)

In [37]:
split(with_mask, with_mask_test_count, with_mask_train_path, with_mask_test_path)
split(without_mask, without_mask_test_count, without_mask_train_path, without_mask_test_path)

In [38]:
experiments_path = '/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset'

In [44]:
with_mask_train_path = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/with_mask/*')
without_mask_train_path = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/train/without_mask/*')

with_mask_test_path = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/with_mask/*')
without_mask_test_path = glob.glob('/content/gdrive/MyDrive/Project/Mask_Detection_Model/dataset/test/without_mask/*')

In [70]:
print(f'with_mask Train : {len(with_mask_train_path)}')
print(f'without_mask Train : {len(without_mask_train_path)}')
print(f'with_mask Test : {len(with_mask_test_path)}')
print(f'without_mask_test : {len(without_mask_test_path)}')

with_mask Train : 3638
without_mask Train : 3719
with_mask Test : 842
without_mask_test : 863


In [46]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]), 
    'test' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [47]:
image_datasets = {x: datasets.ImageFolder(os.path.join(experiments_path, x), data_transforms[x]) for x in ['train', 'test']}

In [48]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                             batch_size=16, 
                                             shuffle=True, 
                                             num_workers=4) 
               for x in ['train', 'test']}

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [49]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f11801c1ee0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f11801c1f40>}

In [50]:
class_names = image_datasets['train'].classes

In [51]:
class_names

['with_mask', 'without_mask']

In [52]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [53]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

In [54]:
print(dataset_sizes)

{'train': 7357, 'test': 1705}


## Model Training

In [55]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()
    best_acc = 0.0
    best_model = copy.deepcopy(model.state_dict())
    
    new_freeze_state = None
    prev_freeze_state = False
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        print('-' * 10)
        
        for phase in ['train', 'test']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
        
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc:{:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())
            
            print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model)
    return model

## MobileNetV2

In [58]:
model_ft = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True).to(device)

# MobileNet Tuning
num_frts = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_frts, len(class_names))

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
# optimizer_ft = optim.Adagrad(model_ft.parameters(), lr=0.001)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [59]:
torchsummary.summary(model_ft, (3, 224, 224), batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [16, 32, 112, 112]             864
       BatchNorm2d-2         [16, 32, 112, 112]              64
             ReLU6-3         [16, 32, 112, 112]               0
            Conv2d-4         [16, 32, 112, 112]             288
       BatchNorm2d-5         [16, 32, 112, 112]              64
             ReLU6-6         [16, 32, 112, 112]               0
            Conv2d-7         [16, 16, 112, 112]             512
       BatchNorm2d-8         [16, 16, 112, 112]              32
  InvertedResidual-9         [16, 16, 112, 112]               0
           Conv2d-10         [16, 96, 112, 112]           1,536
      BatchNorm2d-11         [16, 96, 112, 112]             192
            ReLU6-12         [16, 96, 112, 112]               0
           Conv2d-13           [16, 96, 56, 56]             864
      BatchNorm2d-14           [16, 96,

In [60]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


train Loss: 0.2561 Acc:0.8930



/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.0716 Acc:0.9742

Epoch 1/19
----------
train Loss: 0.1973 Acc:0.9189



...

test Loss: 0.0214 Acc:0.9959

Epoch 18/19
----------
train Loss: 0.0778 Acc:0.9662



/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.0218 Acc:0.9959

Epoch 19/19
----------
train Loss: 0.0809 Acc:0.9651



/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.0215 Acc:0.9947

Training complete in 21.000000m 22s
Best val acc: 0.995894


In [61]:
torch.save(model_ft.to('cpu'), 'mask_model_mobilenet.pth')

In [62]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    #fig = plt.figure(figsize=(10,10))
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            print(preds,"predicitons")
            
            
            for j in range(inputs.size()[0]):
                images_so_far +=1
                #ax = plt.subplot(num_images//len(labels)-1, len(labels), images_so_far)
                #ax.axis('off')
                #ax.set_title('true: {} predicted: {}'.format(class_names[labels[j]], class_names[preds[j]]))
                print('true: {} predicted: {}'.format(class_names[labels[j]], class_names[preds[j]]))
                #imshow(inputs.cpu().data[j])
                
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [63]:
visualize_model(model_ft.to(device))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0], device='cuda:0') predicitons
true: with_mask predicted: with_mask
true: with_mask predicted: with_mask
true: with_mask predicted: with_mask
true: with_mask predicted: with_mask
true: with_mask predicted: with_mask
true: with_mask predicted: with_mask


## ResNet101

In [64]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [74]:
model_ft = models.resnet101(pretrained=True)

num_frts = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_frts, len(class_names))

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
# optimizer_ft = optim.Adagrad(model_ft.parameters(), lr=0.001)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [75]:
import torchsummary
torchsummary.summary(model_ft, (3, 224, 224), batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [16, 64, 112, 112]           9,408
       BatchNorm2d-2         [16, 64, 112, 112]             128
              ReLU-3         [16, 64, 112, 112]               0
         MaxPool2d-4           [16, 64, 56, 56]               0
            Conv2d-5           [16, 64, 56, 56]           4,096
       BatchNorm2d-6           [16, 64, 56, 56]             128
              ReLU-7           [16, 64, 56, 56]               0
            Conv2d-8           [16, 64, 56, 56]          36,864
       BatchNorm2d-9           [16, 64, 56, 56]             128
             ReLU-10           [16, 64, 56, 56]               0
           Conv2d-11          [16, 256, 56, 56]          16,384
      BatchNorm2d-12          [16, 256, 56, 56]             512
           Conv2d-13          [16, 256, 56, 56]          16,384
      BatchNorm2d-14          [16, 256,

In [80]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.0958 Acc:0.9556



/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.0244 Acc:0.9941

Epoch 1/19
----------
train Loss: 0.1026 Acc:0.9523



/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.0253 Acc:0.9935

Epoch 2/19
----------
train Loss: 0.0942 Acc:0.9615



...

test Loss: 0.0245 Acc:0.9947

Epoch 18/19
----------
train Loss: 0.0935 Acc:0.9573



/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.0247 Acc:0.9941

Epoch 19/19
----------
train Loss: 0.0885 Acc:0.9609



/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


test Loss: 0.0254 Acc:0.9941

Training complete in 47.000000m 6s
Best val acc: 0.995308


In [81]:
torch.save(model_ft.to('cpu'), 'mask_model_resnet101.pth')

In [82]:
visualize_model(model_ft.to(device))

tensor([0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0], device='cuda:0') predicitons
true: with_mask predicted: with_mask
true: without_mask predicted: without_mask
true: with_mask predicted: with_mask
true: without_mask predicted: without_mask
true: with_mask predicted: with_mask
true: with_mask predicted: with_mask


In [83]:
!cp /content/mask_model_resnet101.pth /content/gdrive/MyDrive/Project/Mask_Detection_Model
# !cp /content/mask_model_mobilenet.pth /content/gdrive/MyDrive/Project/Mask_Detection_Model